In [2]:
!pip install pydub

In [3]:
import os
import pandas as pd
from tqdm import tqdm
from pydub import AudioSegment
from IPython.display import Audio as AudioI

In [4]:
os.getcwd()

'/root/data/experiments/exp_2023-07-13_chunk_5sec'

In [5]:
os.chdir('../..')

In [6]:
os.getcwd()

'/root/data'

In [7]:
metadata = pd.read_csv('data/metadata-BACKUP.csv')
metadata_train = pd.read_csv('data/train/metadata.csv')
metadata_val = pd.read_csv('data/val/metadata.csv')

In [8]:
#metadata.head()

In [9]:
#metadata_val.head()

In [10]:
new_metadata = pd.DataFrame(columns=['file_name', 'unique_file', 'path', 'species',
                                    'label', 'subset', 'sample_rate', 'num_frames', 'length'])
new_metadata_train_chunked = pd.DataFrame(columns=['file_name', 'label'])
new_metadata_val_chunked = pd.DataFrame(columns=['file_name', 'label'])
new_metadata_test_chunked = pd.DataFrame(columns=['file_name', 'label'])
# new_metadata_val = pd.DataFrame(columns=['file_name', 'label'])
# new_metadata_test = pd.DataFrame(columns=['file_name', 'label'])

In [11]:
threshold = 5
path_metadata = 'data/data_chunks_5sec/'
path_trn_chunked = 'data/data_chunks_5sec/train_chunked/'
path_val_chunked = 'data/data_chunks_5sec/val_chunked/'
path_tst_chunked = 'data/data_chunks_5sec/test_chunked/'
# path_val = 'data/data_chunks_5sec/val/'
# path_tst = 'data/data_chunks_5sec/test/'


In [12]:
for i in tqdm(range(len(metadata))):
    file_name = metadata.iloc[[i]]['file_name'].values[0]
    unique_file = metadata.iloc[[i]]['unique_file'].values[0]
    path = metadata.iloc[[i]]['path'].values[0]
    species = metadata.iloc[[i]]['species'].values[0]
    label = metadata.iloc[[i]]['label'].values[0]
    subset = metadata.iloc[[i]]['subset'].values[0]
    sample_rate = metadata.iloc[[i]]['sample_rate'].values[0]
    num_frames = metadata.iloc[[i]]['num_frames'].values[0]
    length = metadata.iloc[[i]]['length'].values[0]
    audio = AudioSegment.from_wav(path)
    sec = threshold * 1000
    path_chunked = path_trn_chunked if subset=='train' else (path_val_chunked if subset=='validation' else path_tst_chunked) 
    if length > threshold:
        n = int(length//threshold)
        for j in range(0, n+1):
            new_audio = audio[sec*j:sec*(j+1)]
            new_filename = f'{file_name[:-4]}'+'_cut'+str(j)+'.wav'
            new_path = f'{path_chunked+new_filename}'
            new_len = len(new_audio) / 1000 
            new_frames = int(44100 * new_len)
            if new_len >= 1:
                new_audio.export(path_chunked+new_filename, format="wav")
                if subset == 'train':
                    new_metadata_train_chunked.loc[len(new_metadata_train_chunked)] = [new_filename, label]
                elif subset == 'validation':
                    new_metadata_val_chunked.loc[len(new_metadata_val_chunked)] = [new_filename, label] 
                else:
                    new_metadata_test_chunked.loc[len(new_metadata_test_chunked)] = [new_filename, label]                 
                new_path = f'{path_chunked+new_filename}'
                new_row = [new_filename, unique_file, new_path, species, label, subset,
                           sample_rate, new_frames, new_len]
                new_metadata.loc[len(new_metadata)] = new_row
    else:
        audio.export(path_chunked + file_name, format="wav")
        if subset == 'train':
            new_metadata_train_chunked.loc[len(new_metadata_train_chunked)] = [file_name, label]
        elif subset == 'validation':
            new_metadata_val_chunked.loc[len(new_metadata_val_chunked)] = [file_name, label]
        else:
            new_metadata_test_chunked.loc[len(new_metadata_test_chunked)] = [file_name, label] 

        new_path = f'{path + file_name}'
        new_row = [file_name, unique_file, new_path, species, label, subset,
                   sample_rate, num_frames, length]
        new_metadata.loc[len(new_metadata)] = new_row

100%|██████████| 2331/2331 [10:15<00:00,  3.78it/s]


In [13]:
new_metadata_train_chunked.shape, new_metadata_val_chunked.shape, new_metadata.shape

((11910, 2), (3650, 2), (15560, 9))

In [14]:
#metadata[metadata.subset=='train'].length.apply(lambda x: ile(x, 5)).sum(), metadata[metadata.subset=='validation'].length.apply(lambda x: ile(x, 5)).sum()

In [15]:
new_metadata.to_csv(path_metadata + 'metadata.csv', index=False)
new_metadata_train_chunked.to_csv(path_trn_chunked + 'metadata.csv', index=False)
new_metadata_val_chunked.to_csv(path_val_chunked + 'metadata.csv', index=False)

In [53]:
def chunks(path_original, path_chunked, threshold):
    mtd = pd.read_csv(path_original+'metadata.csv')
    label_flag = False
    if 'label' in list(mtd.columns):
        label_flag = True
        
    if label_flag:    
        new_metadata_chunked = pd.DataFrame(columns=['file_name', 'label'])
    else:
        new_metadata_chunked = pd.DataFrame(columns=['file_name'])
    
    
    for i in tqdm(range(len(mtd))):
        file_name = mtd.iloc[[i]]['file_name'].values[0]
        if label_flag:
            label = mtd.iloc[[i]]['label'].values[0]
        audio = AudioSegment.from_wav(path_original+file_name)
        length = len(audio)/1000
        sec = threshold * 1000
        if length > threshold:
            n = int(length//threshold)
            for j in range(0, n+1):
                new_audio = audio[sec*j:sec*(j+1)]
                new_filename = f'{file_name[:-4]}'+'_cut'+str(j)+'.wav'
                new_path = f'{path_chunked+new_filename}'
                new_len = len(new_audio) / 1000 
                if new_len >= 1:
                    new_audio.export(path_chunked+new_filename, format="wav")
                    if label_flag:
                        new_metadata_chunked.loc[len(new_metadata_chunked)] = [new_filename, label]
                    else:
                        new_metadata_chunked.loc[len(new_metadata_chunked)] = [new_filename]                 
        else:
            audio.export(path_chunked + file_name, format="wav")
            if label_flag:
                new_metadata_chunked.loc[len(new_metadata_chunked)] = [file_name, label]
            else:
                new_metadata_chunked.loc[len(new_metadata_chunked)] = [new_filename]            
    return new_metadata_chunked                      

In [54]:
new_metadata_tst_chunked = chunks(path_original='data/test/', path_chunked='data/data_chunks_5sec/test_chunked/', threshold=5)

100%|██████████| 556/556 [01:53<00:00,  4.90it/s]


In [55]:
new_metadata_tst_chunked

,file_name
0,0_cut0.wav
1,0_cut1.wav
2,0_cut1.wav
3,2_cut0.wav
4,2_cut1.wav
...,...
3105,554_cut3.wav
3106,554_cut4.wav
3107,555_cut0.wav
3108,555_cut1.wav


In [56]:
new_metadata_tst_chunked.to_csv(path_tst_chunked + 'metadata.csv', index=False)